In [9]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()


True

In [10]:
#create graph StateGraph
class State(TypedDict):
    messages: Annotated[list, add_messages]
graph = StateGraph(State)

In [11]:
#react agent -> tools
@tool
def get_weather(city: str) -> str:
    """Get the weather in a given city"""
    if city.lower() in ["yorkshire"]:
        return "Its cold and wet"
    else:
        return f"The weather in {city} is sunny"

weather_tool =[get_weather]




In [12]:
#llm model class
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(
    # api_key="sk-proj-1234567890",
    api_key=openai_api_key,
    model="gpt-4o-mini",
    temperature=0.0,
)
#binding tools to llm - tools are now available to the model
llm_with_tools = llm.bind_tools(weather_tool)

In [13]:
#provide tools within graph
#prebuild node provided by langgraph
tool_node = ToolNode(weather_tool)
#add tool node to exisiting grpah
graph.add_node("tool_node", tool_node)

In [14]:
#main entry node to call llm to see if any tools are needed
def prompt_node(state: State) -> State:
    new_message = llm_with_tools.invoke(state["messages"])
    return {"messages": [new_message]}
    #add response to graph state
graph.add_node("prompt_node", prompt_node)



In [15]:
#conditional edge
def condition_edge(state: State) -> Literal["tool_node", '__end__']:
    last_message = state["messages"][-1]
    #check if the last message is a tool call
    if last_message.tool_calls:
        return "tool_node"
    else:
        #if no tool call, end the graph
        return '__end__'
graph.add_conditional_edges("prompt_node", condition_edge)
#add edge from tool node to prompt node
graph.add_edge("tool_node", "prompt_node")
#set entry point to prompt node
graph.set_entry_point("prompt_node")


In [16]:
APP = graph.compile()

#run the graph
result = APP.invoke({"messages": ["What is the weather in Yorkshire?"]})
print(result)

{'messages': [HumanMessage(content='What is the weather in Yorkshire?', additional_kwargs={}, response_metadata={}, id='521ef142-296e-4adc-96b7-e69ee6ab8d4d'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_m4TCaHP88HMtTYvFUoYpDJbb', 'function': {'arguments': '{"city":"Yorkshire"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 52, 'total_tokens': 67, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-Ca586lqtg1KAnB8sluEpudRZrqnnS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a1f66869-4636-444a-82ed-8d7acca3238e-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Yorkshire'}, 'i

In [17]:
print(result["messages"][-1].content)


The weather in Yorkshire is currently cold and wet.
